# SageMaker Neural Topic Model (NTM) Exercise

This notebook demonstrates Amazon SageMaker's **Neural Topic Model (NTM)** algorithm for topic modeling.

## What You'll Learn
1. How to prepare text data for topic modeling
2. How to train an NTM model
3. How to interpret discovered topics

## What is Neural Topic Model?

NTM is an **unsupervised** algorithm that discovers abstract topics in a collection of documents. It uses a neural network approach to learn topic distributions.

**Key Concept:**
- Documents are mixtures of topics
- Topics are distributions over words
- NTM learns both simultaneously

## Use Cases

| Application | Description |
|-------------|-------------|
| Document Organization | Categorize large document collections |
| Content Recommendation | Find similar content by topic |
| Trend Analysis | Track topic evolution over time |
| Search Enhancement | Topic-based document retrieval |
| Customer Feedback | Analyze reviews/surveys by themes |

---

## Step 1: Setup and Imports

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator
import pandas as pd
import numpy as np
import json
import os
import io
from datetime import datetime
from dotenv import load_dotenv
from collections import Counter
import matplotlib.pyplot as plt

# Load environment variables from .env file
load_dotenv()

# Configure AWS session from environment variables
aws_profile = os.getenv('AWS_PROFILE')
aws_region = os.getenv('AWS_REGION', 'us-west-2')
sagemaker_role = os.getenv('SAGEMAKER_ROLE_ARN')

if aws_profile:
    boto3.setup_default_session(profile_name=aws_profile, region_name=aws_region)
else:
    boto3.setup_default_session(region_name=aws_region)

# SageMaker session and role
sagemaker_session = sagemaker.Session()

if sagemaker_role:
    role = sagemaker_role
else:
    role = get_execution_role()

region = sagemaker_session.boto_region_name

print(f"AWS Profile: {aws_profile or 'default'}")
print(f"SageMaker Role: {role}")
print(f"Region: {region}")
print(f"SageMaker SDK Version: {sagemaker.__version__}")

In [ ]:
# Configuration
BUCKET_NAME = sagemaker_session.default_bucket()
PREFIX = "neural-topic-model"

# Dataset parameters
NUM_DOCUMENTS = 1000
NUM_TOPICS = 5
VOCAB_SIZE = 500
RANDOM_STATE = 42

print(f"S3 Bucket: {BUCKET_NAME}")
print(f"S3 Prefix: {PREFIX}")

## Step 2: Generate Synthetic Document Data

In [ ]:
def generate_topic_documents(num_docs=1000, num_topics=5, seed=42):
    """
    Generate synthetic documents with known topic structure.
    
    Each topic has a vocabulary of associated words.
    Documents are mixtures of topics.
    """
    np.random.seed(seed)
    
    # Define topic vocabularies
    topic_words = {
        0: ['technology', 'software', 'computer', 'data', 'internet', 'digital', 
            'programming', 'algorithm', 'system', 'application', 'code', 'developer',
            'network', 'cloud', 'security', 'database', 'server', 'api', 'machine', 'learning'],
        1: ['sports', 'team', 'game', 'player', 'championship', 'score', 
            'win', 'match', 'coach', 'season', 'league', 'tournament',
            'athlete', 'training', 'competition', 'stadium', 'fans', 'victory', 'goal', 'record'],
        2: ['health', 'medical', 'patient', 'doctor', 'treatment', 'disease', 
            'hospital', 'medicine', 'research', 'clinical', 'therapy', 'diagnosis',
            'symptoms', 'care', 'wellness', 'prevention', 'vaccine', 'nutrition', 'exercise', 'mental'],
        3: ['business', 'market', 'company', 'financial', 'investment', 'economy', 
            'growth', 'revenue', 'profit', 'stock', 'industry', 'management',
            'strategy', 'customer', 'sales', 'startup', 'enterprise', 'capital', 'trade', 'commerce'],
        4: ['science', 'research', 'study', 'experiment', 'discovery', 'theory', 
            'physics', 'chemistry', 'biology', 'laboratory', 'scientist', 'analysis',
            'hypothesis', 'evidence', 'findings', 'nature', 'universe', 'energy', 'molecule', 'observation']
    }
    
    topic_names = ['Technology', 'Sports', 'Health', 'Business', 'Science']
    
    # Build vocabulary
    all_words = []
    for words in topic_words.values():
        all_words.extend(words)
    vocab = sorted(set(all_words))
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    
    documents = []
    doc_topics = []  # Ground truth dominant topic
    
    for _ in range(num_docs):
        # Sample dominant topic
        dominant_topic = np.random.randint(0, num_topics)
        doc_topics.append(dominant_topic)
        
        # Generate document length
        doc_length = np.random.randint(50, 150)
        
        # Generate words (80% from dominant topic, 20% from others)
        doc_words = []
        for _ in range(doc_length):
            if np.random.random() < 0.8:
                word = np.random.choice(topic_words[dominant_topic])
            else:
                other_topic = np.random.choice([t for t in range(num_topics) if t != dominant_topic])
                word = np.random.choice(topic_words[other_topic])
            doc_words.append(word)
        
        documents.append(' '.join(doc_words))
    
    return documents, doc_topics, vocab, word_to_idx, topic_names

# Generate documents
documents, doc_topics, vocab, word_to_idx, topic_names = generate_topic_documents(
    NUM_DOCUMENTS, NUM_TOPICS, RANDOM_STATE
)

print(f"Generated {len(documents)} documents")
print(f"Vocabulary size: {len(vocab)}")
print(f"Topics: {topic_names}")
print(f"\nTopic distribution: {Counter(doc_topics)}")

print(f"\nSample document (Topic: {topic_names[doc_topics[0]]}):")
print(f"  {documents[0][:200]}...")

## Step 3: Prepare Data for NTM

NTM expects **bag-of-words** representation:
- Each document is a vector of word counts
- Vector length = vocabulary size
- Format: CSV or RecordIO-protobuf

In [ ]:
def documents_to_bow(documents, word_to_idx):
    """
    Convert documents to bag-of-words matrix.
    
    Returns:
        bow_matrix: Shape (num_docs, vocab_size)
    """
    vocab_size = len(word_to_idx)
    bow_matrix = np.zeros((len(documents), vocab_size), dtype=np.float32)
    
    for doc_idx, doc in enumerate(documents):
        words = doc.lower().split()
        for word in words:
            if word in word_to_idx:
                bow_matrix[doc_idx, word_to_idx[word]] += 1
    
    return bow_matrix

# Convert to bag-of-words
bow_matrix = documents_to_bow(documents, word_to_idx)

print(f"Bag-of-words matrix shape: {bow_matrix.shape}")
print(f"Non-zero entries: {np.count_nonzero(bow_matrix)}")
print(f"Sparsity: {1 - np.count_nonzero(bow_matrix) / bow_matrix.size:.4f}")

In [ ]:
# Split into train and validation
np.random.seed(RANDOM_STATE)
indices = np.random.permutation(len(bow_matrix))

val_size = int(0.1 * len(bow_matrix))
train_idx = indices[val_size:]
val_idx = indices[:val_size]

train_data = bow_matrix[train_idx]
val_data = bow_matrix[val_idx]

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

In [ ]:
# Save as CSV
os.makedirs('data/ntm', exist_ok=True)

np.savetxt('data/ntm/train.csv', train_data, delimiter=',')
np.savetxt('data/ntm/validation.csv', val_data, delimiter=',')

# Save vocabulary for later interpretation
with open('data/ntm/vocab.json', 'w') as f:
    json.dump({'vocab': vocab, 'word_to_idx': word_to_idx}, f)

print("Data files created:")
for f in os.listdir('data/ntm'):
    size = os.path.getsize(f'data/ntm/{f}') / 1024
    print(f"  data/ntm/{f} ({size:.1f} KB)")

In [ ]:
# Upload to S3
s3_client = boto3.client('s3')

for split in ['train', 'validation']:
    s3_key = f"{PREFIX}/{split}/{split}.csv"
    s3_client.upload_file(f'data/ntm/{split}.csv', BUCKET_NAME, s3_key)
    print(f"Uploaded: s3://{BUCKET_NAME}/{s3_key}")

train_uri = f"s3://{BUCKET_NAME}/{PREFIX}/train"
val_uri = f"s3://{BUCKET_NAME}/{PREFIX}/validation"

## Step 4: Train NTM Model

### Key Hyperparameters

| Parameter | Description | Default |
|-----------|-------------|---------|
| `num_topics` | Number of topics to discover | Required |
| `feature_dim` | Vocabulary size | Required |
| `mini_batch_size` | Batch size | 256 |
| `epochs` | Training epochs | 50 |
| `learning_rate` | Learning rate | 0.001 |
| `encoder_layers` | Hidden layer sizes | [100] |
| `encoder_layers_activation` | Activation function | relu |

In [ ]:
# Get NTM container image
ntm_image = retrieve(
    framework='ntm',
    region=region,
    version='1'
)

print(f"NTM Image URI: {ntm_image}")

In [ ]:
# Create NTM estimator
ntm_estimator = Estimator(
    image_uri=ntm_image,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{BUCKET_NAME}/{PREFIX}/output',
    sagemaker_session=sagemaker_session,
    base_job_name='neural-topic-model'
)

In [ ]:
# Set hyperparameters
hyperparameters = {
    "num_topics": NUM_TOPICS,
    "feature_dim": len(vocab),
    "mini_batch_size": 128,
    "epochs": 50,
    "learning_rate": 0.002,
    "optimizer": "adam",
    "num_patience_epochs": 5,
    "tolerance": 0.001,
}

ntm_estimator.set_hyperparameters(**hyperparameters)

print("NTM hyperparameters:")
for k, v in hyperparameters.items():
    print(f"  {k}: {v}")

In [ ]:
# Start training
print("Starting NTM training job...")
print("This will take approximately 5-10 minutes.\n")

ntm_estimator.fit(
    {
        'train': train_uri,
        'validation': val_uri
    },
    wait=True,
    logs=True
)

In [ ]:
# Get training job info
job_name = ntm_estimator.latest_training_job.name
print(f"Training job completed: {job_name}")
print(f"Model artifacts: {ntm_estimator.model_data}")

## Step 5: Deploy and Get Topic Distributions

In [ ]:
# Deploy the model
print("Deploying NTM model...")
print("This will take approximately 5-7 minutes.\n")

ntm_predictor = ntm_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name=f'ntm-{datetime.now().strftime("%Y%m%d%H%M")}'
)

print(f"\nEndpoint deployed: {ntm_predictor.endpoint_name}")

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Configure predictor
ntm_predictor.serializer = CSVSerializer()
ntm_predictor.deserializer = JSONDeserializer()

def get_topic_distributions(data, predictor, batch_size=100):
    """
    Get topic distributions for documents.
    """
    all_distributions = []
    
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        response = predictor.predict(batch)
        
        for pred in response['predictions']:
            all_distributions.append(pred['topic_weights'])
    
    return np.array(all_distributions)

In [ ]:
# Get topic distributions for all documents
print("Getting topic distributions...")
topic_distributions = get_topic_distributions(bow_matrix, ntm_predictor)

print(f"Topic distributions shape: {topic_distributions.shape}")
print(f"\nSample distribution (first document):")
for i, weight in enumerate(topic_distributions[0]):
    print(f"  Topic {i}: {weight:.4f}")

## Step 6: Analyze Discovered Topics

In [ ]:
# Assign documents to their dominant topic
predicted_topics = np.argmax(topic_distributions, axis=1)

print("Predicted Topic Distribution:")
print(Counter(predicted_topics))

print("\nTrue Topic Distribution:")
print(Counter(doc_topics))

In [ ]:
# Visualize topic distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Average topic weights
avg_weights = topic_distributions.mean(axis=0)
axes[0].bar(range(NUM_TOPICS), avg_weights)
axes[0].set_xlabel('Topic')
axes[0].set_ylabel('Average Weight')
axes[0].set_title('Average Topic Weights Across All Documents')
axes[0].set_xticks(range(NUM_TOPICS))

# Topic distribution heatmap for sample documents
sample_docs = np.random.choice(len(topic_distributions), 20, replace=False)
im = axes[1].imshow(topic_distributions[sample_docs], aspect='auto', cmap='YlOrRd')
axes[1].set_xlabel('Topic')
axes[1].set_ylabel('Document')
axes[1].set_title('Topic Weights for Sample Documents')
axes[1].set_xticks(range(NUM_TOPICS))
plt.colorbar(im, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# Compare predicted vs true topics (matching by most common co-occurrence)
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(doc_topics, predicted_topics)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=[f'Pred {i}' for i in range(NUM_TOPICS)],
            yticklabels=topic_names)
ax.set_xlabel('Predicted Topic')
ax.set_ylabel('True Topic')
ax.set_title('True vs Predicted Topic Confusion Matrix')
plt.tight_layout()
plt.show()

## Step 7: Interpret Topics by Top Words

To interpret topics, we examine which words have highest probability in each topic.

In [ ]:
# For each discovered topic, find documents most representative
# and examine their most frequent words

print("Top Words per Discovered Topic (based on document analysis):")
print("=" * 60)

for topic_id in range(NUM_TOPICS):
    # Get documents where this topic is dominant
    topic_docs_mask = predicted_topics == topic_id
    topic_docs = bow_matrix[topic_docs_mask]
    
    # Sum word counts across documents
    word_counts = topic_docs.sum(axis=0)
    
    # Get top words
    top_word_indices = np.argsort(word_counts)[::-1][:10]
    top_words = [vocab[idx] for idx in top_word_indices]
    
    print(f"\nTopic {topic_id} ({topic_docs_mask.sum()} documents):")
    print(f"  Top words: {', '.join(top_words)}")

## Step 8: Clean Up Resources

In [ ]:
# Delete the endpoint
print(f"Deleting endpoint: {ntm_predictor.endpoint_name}")
ntm_predictor.delete_endpoint()
print("Endpoint deleted successfully!")

---

## Summary

In this exercise, you learned:

1. **Data Format**: Bag-of-words vectors (CSV or RecordIO)

2. **Key Hyperparameters**:
   - `num_topics`: Number of topics to discover
   - `feature_dim`: Vocabulary size
   - `epochs`, `learning_rate`: Training parameters

3. **Output**: Topic distribution vector per document

4. **Topic Interpretation**:
   - Examine top words per topic
   - Review representative documents
   - Use domain knowledge to label topics

### NTM vs LDA

| Aspect | NTM | LDA |
|--------|-----|-----|
| Architecture | Neural network | Probabilistic |
| Training | GPU supported | CPU only |
| Scalability | Multi-GPU, distributed | Single CPU |
| Coherence | Good | Better (theory) |
| Speed | Faster | Slower |

### Instance Recommendations

| Task | Instance Types |
|------|----------------|
| Training | ml.c5.xlarge, ml.p2.xlarge, ml.p3.2xlarge |
| Inference | ml.m5.large, ml.c5.large |

### Next Steps

- Apply to real document collections
- Experiment with different `num_topics` values
- Use topic distributions as features for downstream tasks
- Combine with document embeddings for richer representations